### Create Spark Context

In [1]:
from pyspark.sql import SparkSession
import time as t

In [2]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

In [3]:
spark

### Download JDBC driver and put it into extensions folder of JAVA

In [4]:
jdbcHostname = "192.168.0.169"
jdbcDatabase = "PSX"
username = "SA"
password = "Yasin@0334"
jdbcUrl = "jdbc:sqlserver://{0};database={1};user={2};password={3}".format(jdbcHostname, jdbcDatabase, 
                                                                           username, password)
connectionProperties = {
  "driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [5]:
pushdown_query = "(select * from universe where ticker = 'SEARL') uni"
df = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)

Py4JJavaError: An error occurred while calling o30.jdbc.
: com.microsoft.sqlserver.jdbc.SQLServerException: The TCP/IP connection to the host 192.168.0.169, port 1433 has failed. Error: "Host is down (connect failed). Verify the connection properties. Make sure that an instance of SQL Server is running on the host and accepting TCP/IP connections at the port. Make sure that TCP connections to the port are not blocked by a firewall.".
	at com.microsoft.sqlserver.jdbc.SQLServerException.makeFromDriverError(SQLServerException.java:234)
	at com.microsoft.sqlserver.jdbc.SQLServerException.ConvertConnectExceptionToSQLServerException(SQLServerException.java:285)
	at com.microsoft.sqlserver.jdbc.SocketFinder.findSocket(IOBuffer.java:2431)
	at com.microsoft.sqlserver.jdbc.TDSChannel.open(IOBuffer.java:656)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.connectHelper(SQLServerConnection.java:2472)
	at com.microsoft.sqlserver.jdbc.SQLServerConnection.login(SQLServerConnection.java:2142)
	at

In [ ]:
df.show()

### Get table Schema

In [ ]:
df.printSchema()

### Describe DataFrame

In [ ]:
df.describe().show()

### Get Column Names

In [ ]:
df.columns

### Filter DataFrame to get Column or DataFrame object

#### Column Object

In [ ]:
type(df['ticker'])

#### DataFrame Object

In [ ]:
type(df.select('ticker'))

##### Get Multiple Columns

In [ ]:
df['ticker', 'Open']

In [ ]:
df.select(['ticker', 'Open']).show(2)

In [ ]:
type(df.select(['ticker', 'Open']))

### Get DF Rows

In [ ]:
df.head(2)[0]

In [ ]:
type(df.head(2)[0])

### Add new column to DataFrame

In [ ]:
df.withColumn('AOHLC', (df['Open'] + df['High'] + df['Low'] + df['Close'])/4).show(2)

### Above addition of column was not permenant. See below

In [ ]:
df.show(2)

### Assign new df to incorporate changes

In [ ]:
df_ohlc = df.withColumn('AOHLC', (df['Open'] + df['High'] + df['Low'] + df['Close'])/4)

In [ ]:
df_ohlc.show(2)

### Rename Columns

#### Column Renaming is not permenant

In [ ]:
df_ohlc.withColumnRenamed('AOHLC', 'Average_ohlc').show(2)

In [ ]:
df_ohlc.show(2)

### Register DataFrame as SQL TempView
#### i) SQL can be used to query a dataframe 

In [ ]:
df_ohlc.createOrReplaceTempView('prices')

In [ ]:
open_price = spark.sql('SELECT Date, Open FROM PRICES')

In [ ]:
open_price.show(2)

### Filter DataFrame

In [ ]:
df.filter("Close < 100").show(2)

In [ ]:
df.filter((df['Close'] < 100) 
          & (df['Open'] > 100)).show(2)

### Collecting Results

In [ ]:
close_100 = df.filter((df['Close'] < 100) 
          & (df['Open'] > 100)).collect()

#### Get all the records as list of rows

In [ ]:
close_100

### Groupby 

In [ ]:
df.groupBy('ticker').max().show()

In [ ]:
df.agg({'Open':'max'}).show()

In [ ]:
df.agg({'Open': 'min'}).show()

In [ ]:
pushdown_query = "(select * from universe where ticker IN ('SEARL', 'OGDC')) uni"
df_mult_tick = spark.read.jdbc(url=jdbcUrl, table=pushdown_query, properties=connectionProperties)

#### Groupby Object

In [ ]:
df_groupby_obj = df_mult_tick.groupBy('ticker')

In [ ]:
df_groupby_obj.agg({'open':'max'}).show()

In [ ]:
df_mult_tick.groupBy('ticker').agg({'open':'min'}).show()

### Pyspark Functions 

In [ ]:
from pyspark.sql.functions import avg,countDistinct,stddev,count,format_number

In [ ]:
df_mult_tick.filter(df_mult_tick.ticker == 'OGDC').select(avg('High').alias('Average OGDC')) \
.select(format_number('Average OGDC', 2).alias('Avg ODGDC 2 DEC')).show()

In [ ]:
df_mult_tick.filter(df_mult_tick.ticker == 'OGDC').select(avg('High').alias('Average OGDC')) \
.select(format_number('Average OGDC', 3).alias('Avg ODGDC 3 DEC')).show()

### Dealing with Decimal places

In [ ]:
df_mult_tick.filter(df_mult_tick.ticker == 'OGDC').select(avg('High').alias('Average OGDC')).show()

#### Other Functions

In [ ]:
df_mult_tick.filter(df_mult_tick.ticker == 'OGDC').select(count('High').alias('Count OGDC')).show()

In [ ]:
df_mult_tick.filter(df_mult_tick.ticker == 'OGDC').select(countDistinct('High').alias('Distinct Count OGDC')).show()

### OrderBy 

In [ ]:
df_mult_tick.orderBy('Date').show()

In [ ]:
df_mult_tick.orderBy(df_mult_tick['Date'].desc()).show()

### Dealing With Null Values

In [ ]:
null_df = spark.read.csv('NullData.csv', header=True)

In [ ]:
null_df.printSchema()

In [ ]:
null_df.show()

#### Drop NA Values

In [ ]:
null_df.na.drop().show()

#### Drop NA Values with threeshold

In [ ]:
# Each row must contain atleast 2 non null values to be not dropped
null_df.na.drop(thresh =2).show()

In [ ]:
# define subset to drop
null_df.na.drop(subset = ['Debit']).show()

In [ ]:
from pyspark.sql.types import FloatType, IntegerType

In [ ]:
converted_null_df = null_df.withColumn('Debit', null_df['Debit'].cast(IntegerType()))

In [ ]:
converted_null_df = converted_null_df.withColumn('Credit', converted_null_df['Credit'].cast(IntegerType()))

In [ ]:
converted_null_df.show()

In [ ]:
converted_null_df.groupby('Acct').pivot('Month').sum('Debit').show()

In [ ]:
pivoted_data = converted_null_df.groupby('Acct').pivot('Month').sum('Debit', 'Credit')

In [ ]:
##Renaming columns in above df
#https://sparkbyexamples.com/spark/rename-a-column-on-spark-dataframes/
renamed_conv_null_df = pivoted_data.withColumnRenamed('1_sum(CAST(Debit AS BIGINT))', 'M1D') \
                      .withColumnRenamed('1_sum(CAST(Credit AS BIGINT))', 'M1C') \
                      .withColumnRenamed('2_sum(CAST(Debit AS BIGINT))', 'M2D') \
                    .withColumnRenamed('2_sum(CAST(Credit AS BIGINT))', 'M2C') \
                    .withColumnRenamed('3_sum(CAST(Debit AS BIGINT))', 'M3D') \
                    .withColumnRenamed('3_sum(CAST(Credit AS BIGINT))', 'M3C') 

In [ ]:
pivoted_data.printSchema()

In [ ]:
renamed_conv_null_df.show()

In [ ]:
from pyspark.sql.functions import *

In [ ]:
renamed_conv_null_df.select('M2D').na.fill(0)(when(col('M2D') == 0, 1).otherwise('M2D'))

In [ ]:
#https://stackoverflow.com/questions/39048229/spark-equivalent-of-if-then-else
renamed_conv_null_df.select('M2D').na.fill(0) \
.withColumn('M2D', when(col('M2D') == 0, 1) \
            .otherwise(col('M2D'))).show()

In [ ]:
renamed_conv_null_df.show()

In [ ]:
#https://blog.justalfred.com/html/PyGotham2016.html
renamed_conv_null_df = renamed_conv_null_df.withColumn('Ratio', 
                                                      renamed_conv_null_df.select('M2C').na.fill(0))



In [ ]:
type(renamed_conv_null_df.select('M2D').na.fill(0) \
.withColumn('M2D', when(col('M2D') == 0, 1) \
.otherwise(col('M2D')))['M2D'])

In [ ]:
type(renamed_conv_null_df.select('M2C').na.fill(0)['M2C'])